<a href="https://colab.research.google.com/github/sreenithya2118/House-pricing/blob/main/majorproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import time
import numpy as np
import random

# Define Cloud Tasks with task number and descriptions
tasks = [
    {"task_number": 1, "description": "Deploy Cloud Application on AWS", "priority": 1, "due_date": 1680240000, "estimated_time": 3},
    {"task_number": 2, "description": "Provision Virtual Machines on GCP", "priority": 2, "due_date": 1680245000, "estimated_time": 2},
    {"task_number": 3, "description": "Set Up Cloud Load Balancer on Azure", "priority": 3, "due_date": 1680250000, "estimated_time": 1},
    {"task_number": 4, "description": "Set Up Cloud Auto-Scaling on AWS", "priority": 1, "due_date": 1680255000, "estimated_time": 4},
    {"task_number": 5, "description": "Cloud Database Setup & Optimization on GCP", "priority": 4, "due_date": 1680260000, "estimated_time": 2},
    {"task_number": 6, "description": "Configure Cloud Network on Azure", "priority": 2, "due_date": 1680265000, "estimated_time": 3},
    {"task_number": 7, "description": "Implement Cloud Security on AWS", "priority": 3, "due_date": 1680270000, "estimated_time": 3},
    {"task_number": 8, "description": "Monitor Cloud Resources on GCP", "priority": 5, "due_date": 1680275000, "estimated_time": 1},
    {"task_number": 9, "description": "Set Up Cloud Disaster Recovery on Azure", "priority": 3, "due_date": 1680280000, "estimated_time": 4},
    {"task_number": 10, "description": "Migrate Data to Cloud Storage on AWS", "priority": 4, "due_date": 1680285000, "estimated_time": 2},
    {"task_number": 11, "description": "Configure Cloud CDN on Azure", "priority": 2, "due_date": 1680290000, "estimated_time": 3},
    {"task_number": 12, "description": "Optimize Cloud Costs on AWS", "priority": 5, "due_date": 1680295000, "estimated_time": 1},
    {"task_number": 13, "description": "Configure Cloud IAM on GCP", "priority": 1, "due_date": 1680300000, "estimated_time": 2},
    {"task_number": 14, "description": "Set Up Cloud Logging & Auditing on AWS", "priority": 4, "due_date": 1680305000, "estimated_time": 2}
]

# Blockchain Class for storing tasks and ensuring immutability
class Blockchain:
    def __init__(self):
        self.chain = []
        self.create_genesis_block()

    def create_genesis_block(self):
        """Create the first block in the blockchain (genesis block)."""
        genesis_block = self.create_block(previous_hash="0")
        self.chain.append(genesis_block)

    def create_block(self, previous_hash):
        """Create a new block containing the current task information."""
        block = {
            'timestamp': time.time(),
            'task_number': None,
            'task_description': None,
            'reward': None,
            'previous_hash': previous_hash,
            'hash': None
        }
        block['hash'] = self.hash_block(block)
        return block

    def hash_block(self, block):
        """Generate hash of the block using SHA256."""
        block_string = str(block['timestamp']) + str(block['task_number']) + str(block['task_description']) + str(block['reward']) + str(block['previous_hash'])
        return hashlib.sha256(block_string.encode('utf-8')).hexdigest()

    def add_block(self, task, reward):
        """Add a new block to the blockchain."""
        last_block = self.chain[-1]
        block = self.create_block(last_block['hash'])
        block['task_number'] = task['task_number']
        block['task_description'] = task['description']
        block['reward'] = reward
        self.chain.append(block)

    def get_chain(self):
        """Get the blockchain data."""
        return self.chain

# DRL Agent Class
class DRLAgent:
    def __init__(self, tasks, n_actions, blockchain):
        self.tasks = tasks
        self.n_actions = n_actions
        self.q_table = np.zeros((len(tasks), len(tasks)))  # Initialize Q-table for task scheduling
        self.learning_rate = 0.1
        self.discount_factor = 0.9
        self.exploration_rate = 1.0
        self.exploration_decay = 0.995
        self.blockchain = blockchain  # Add blockchain to the DRL agent

    def calculate_reward(self, task):
        """Calculate reward dynamically based on task properties"""
        reward = task['priority'] * 50
        reward -= task['estimated_time'] * 10
        reward += max(0, 100 - (task['due_date'] - 1680240000) // 100000)
        return reward

    def choose_task_to_complete(self, incomplete_tasks):
        """Choose task based on exploration/exploitation strategy"""
        if random.random() < self.exploration_rate:
            return random.choice(incomplete_tasks)
        else:
            task_q_values = [self.q_table[task['task_number'] - 1] for task in incomplete_tasks]
            best_task_index = np.argmax(task_q_values)
            return incomplete_tasks[best_task_index]

    def update_q_table(self, task_id, reward, next_task_id):
        """Update the Q-table using Q-learning update rule"""
        old_q_value = self.q_table[task_id]
        future_reward = np.max(self.q_table[next_task_id])
        new_q_value = old_q_value + self.learning_rate * (reward + self.discount_factor * future_reward - old_q_value)
        self.q_table[task_id] = new_q_value

    def schedule_tasks(self):
        scheduled_tasks = []
        incomplete_tasks = self.tasks.copy()  # Start with all tasks as incomplete

        while incomplete_tasks:
            task_to_schedule = self.choose_task_to_complete(incomplete_tasks)
            task_id = task_to_schedule['task_number'] - 1  # Convert task number to index
            reward_points = self.calculate_reward(task_to_schedule)

            scheduled_tasks.append((task_to_schedule, reward_points))
            incomplete_tasks.remove(task_to_schedule)

            # Simulate the update of Q-table
            if incomplete_tasks:
                next_task_to_schedule = incomplete_tasks[0]
                next_task_id = next_task_to_schedule['task_number'] - 1
                self.update_q_table(task_id, reward_points, next_task_id)

            # Add the task to the blockchain after completion
            self.blockchain.add_block(task_to_schedule, reward_points)

            # Decay the exploration rate
            self.exploration_rate *= self.exploration_decay

        return scheduled_tasks

# Instantiate Blockchain and DRL agent
blockchain = Blockchain()
drl_agent = DRLAgent(tasks=tasks, n_actions=5, blockchain=blockchain)

# Schedule tasks using DRL agent logic
scheduled_tasks = drl_agent.schedule_tasks()

# Print added tasks (without reward points)
print("Added Tasks (Original Order):")
for task in tasks:
    print(f"Task {task['task_number']}: {task['description']}")

print("\n")

# Print scheduled tasks (with dynamic reward points)
print("Scheduled Tasks by DRL:")
for task, reward in scheduled_tasks:
    print(f"Task {task['task_number']}: {task['description']} - Reward: {reward} points")

# Print blockchain data
print("\nBlockchain Ledger:")
for block in blockchain.get_chain():
    print(block)

Added Tasks (Original Order):
Task 1: Deploy Cloud Application on AWS
Task 2: Provision Virtual Machines on GCP
Task 3: Set Up Cloud Load Balancer on Azure
Task 4: Set Up Cloud Auto-Scaling on AWS
Task 5: Cloud Database Setup & Optimization on GCP
Task 6: Configure Cloud Network on Azure
Task 7: Implement Cloud Security on AWS
Task 8: Monitor Cloud Resources on GCP
Task 9: Set Up Cloud Disaster Recovery on Azure
Task 10: Migrate Data to Cloud Storage on AWS
Task 11: Configure Cloud CDN on Azure
Task 12: Optimize Cloud Costs on AWS
Task 13: Configure Cloud IAM on GCP
Task 14: Set Up Cloud Logging & Auditing on AWS


Scheduled Tasks by DRL:
Task 3: Set Up Cloud Load Balancer on Azure - Reward: 240 points
Task 8: Monitor Cloud Resources on GCP - Reward: 340 points
Task 1: Deploy Cloud Application on AWS - Reward: 120 points
Task 7: Implement Cloud Security on AWS - Reward: 220 points
Task 9: Set Up Cloud Disaster Recovery on Azure - Reward: 210 points
Task 10: Migrate Data to Cloud Storag